In [13]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
import warnings
warnings.filterwarnings("ignore")
import os
import pathlib
import random 
import argparse
from scipy import optimize

inputdir = "/Volumes/HNWD02/outdir/output_methyl_ReadBased_Models/01_output/TSMA_panel"
path_to_all_fa = "/Volumes/HNSD01/storage/ref/hg19"
outdir = "/Volumes/HNSD01/outdir"
outputdir = os.path.join(outdir, "output_methyl_ReadBased_Models")
path_to_01_output = os.path.join(outputdir, "01_output")
path_to_02_output = os.path.join(outputdir, "02_output")

metadata = pd.read_excel("/Users/hieunguyen/src/tsma_micro_atlas/metadata/metadata_TSMA_R8281_R8282.xlsx")
paneldf = pd.read_excel("./assets/12967_2024_5416_MOESM1_ESM.xlsx")
all_regions = paneldf.Region_name.unique()

path_to_03_output = os.path.join(outputdir, "03_output")
path_to_03a_output = os.path.join(outputdir, "03_output")
os.system(f"mkdir -p {path_to_03a_output}")

sampleid = "LAAQ91TS"
all_preds = [
        item for item in pathlib.Path(path_to_03_output).glob(f"*{sampleid}/microAtlas_data/region_*/*_methyl_pattern_by_tissue_type.csv")
    ]

In [36]:
cosdf = pd.DataFrame()
for file in tqdm(all_preds):
    region_name = file.name.replace("_methyl_pattern_by_tissue_type.csv", "")
    tmpdf = pd.read_csv(file, index_col=0)

    from sklearn.metrics.pairwise import cosine_similarity

    cosine_sim = cosine_similarity(tmpdf.T)
    cosine_sim_df = pd.DataFrame(cosine_sim, 
                                index=tmpdf.columns, 
                                columns=tmpdf.columns)
    sum_cos_sim = np.sum(cosine_sim_df.abs().sum().sort_values(ascending=False) - 1)
    tmpdf = pd.DataFrame({
        "region_name": region_name,
        "sum_cosine_similarity": sum_cos_sim
    }, index=[0])
    cosdf = pd.concat([cosdf, tmpdf], ignore_index=True)


100%|██████████| 2541/2541 [00:03<00:00, 796.54it/s]


In [40]:
cosine_sim_df

,Gastric,Lung
Gastric,1.0,1.0
Lung,1.0,1.0


In [38]:
cosdf.sort_values(by="sum_cosine_similarity", ascending=False, inplace=True)

In [42]:
cosdf[cosdf["sum_cosine_similarity"] <= 0]

,region_name,sum_cosine_similarity
13,20_30640325_30640427,0.000000e+00
2359,5_783046_783140,0.000000e+00
953,1_109288966_109289066,0.000000e+00
2521,2_241497106_241497208,0.000000e+00
1026,8_145747871_145747967,0.000000e+00
...,...,...
2069,5_72415987_72416082,-4.440892e-16
593,11_93474118_93474198,-4.440892e-16
921,5_115152170_115152258,-4.440892e-16
346,16_88600520_88600622,-4.440892e-16
